### Life Cycle Management Data Analysis 🌽
Considerations:
- This notebook is intended for the analysis of Double Haploid Life Cycle Management experiments developed in GTES between 2021 and 2024
- The dataset used is `full_lcm_data.csv` obtained from the notebook `DH LCM Data Merger`
- The unique identifier for an Induction Plant is 'IND-ID' column.
- Dataset columns are grouped by 4 main processes, which are named as prefix in each column (__*IND*__: Induction, __*LAB*__: Laboratory, __*HRD*__: Hardening, __*BN*__: Baby Nursery)
- Datapoints of Hardening and Baby Nursery have relationship *many-to-one* with Induction and Laboratory datapoints, which were repeated to fill blank values.
- This means that in __Induction and Laboratory datapoints, each plant is repeated many times, and not always the same number of times. Need to select unique values of IND-ID for a proper analysis of corresponding columns.__

In [ ]:
#Function for installing dependencies from requirements.txt
import subprocess
import sys
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn import preprocessing
import statsmodels.api as sm
import itertools
def install_requirements(file_path):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', file_path])

#if libraries not installed, execute the code below this comment    
#install_requirements('requirements.txt')

In [ ]:
#Fetching data from csv
full_data = pd.read_csv('full_lcm_data.csv', low_memory=False)
data_info = full_data.info()
data_info

In [ ]:
#Quick overview of dataset
unique_ind_data = full_data.drop_duplicates(subset=['IND-ID'])
total_ind_plants = full_data['IND-ID'].nunique()
first_sow = full_data['IND_Sowing_Date'].min()
last_sow = full_data['IND_Sowing_Date'].max()
germinated_plants = unique_ind_data[unique_ind_data['IND_%Germin.']==1]['IND-ID'].nunique()
pollinated_plants = unique_ind_data[unique_ind_data['IND_%Pollination']==1]['IND-ID'].nunique()
ears_harvested = unique_ind_data[unique_ind_data['IND_Ear_harvested']==True]['IND-ID'].nunique()
material_count = full_data['IND_Induction_MATID'].nunique()
total_embryos = unique_ind_data['LAB_#Total_Embryos'].sum()
haploid_embryos = unique_ind_data['LAB_#Haploids_Embryos'].sum()
selected_seedlings = unique_ind_data['LAB_Total_selected_seedlings'].sum()
print(f'**HOW MANY INDUCTION PLANTS ARE IN THIS DATASET?** \nTotal Materials: {material_count} different materials')
print(f'- {total_ind_plants} Induction plants were sowed (first sow on {first_sow}, last sow on {last_sow})\n- {germinated_plants} Induction plants germinated (Germination rate: {germinated_plants/total_ind_plants:.2%})')
print(f'- {pollinated_plants} Induction plants successfully pollinated (Pollination rate: {pollinated_plants/germinated_plants:.2%})')
print(f'- {ears_harvested} Induction plants successfully harvested (Harvest rate: {ears_harvested/pollinated_plants:.2%})')
print(f'- {int(total_embryos)} Embryos processed ({int(haploid_embryos)} Haploids, Haploidy rate: {haploid_embryos/total_embryos:.2%} )')
print(f'- {int(selected_seedlings)} Seedlings selected')
full_data.groupby(['IND_Planting_number','IND_Induction_MATID'])['IND_Induction_MATID'].count()

In [ ]:
#Identification and visualization of significant correlations using Pearson's correlation coefficient
#Calculation is done in every combination possible for all numeric columns, 
#saves in a table the significant correlations and create a dispersion chart
correlation_data = []
float_columns = full_data.select_dtypes(include=['float64']).columns
color_segmentation = 'IND_Planting_number'
mapping = {
    '1st Planting': 1, 
    '2nd Planting': 2, 
    '3rd Planting': 3, 
    '4th Planting': 4,
    '5th Planting': 5, 
    '6th Planting': 6, 
    '7th Planting': 7, 
    '8th Planting': 8,
    '9th Planting': 9, 
    '10th Planting': 10, 
    '11th Planting': 11, 
    '12th Planting': 12,
    '13th Planting': 13}  

full_data['encoded_planting_number'] = full_data['IND_Planting_number'].map(mapping)
num_colors = len(full_data['encoded_planting_number'].unique())
cmap = plt.cm.get_cmap('jet', num_colors)

for column1 in float_columns:
    for column2 in float_columns:
        if column1 != column2:
            correlation = full_data[column1].corr(full_data[column2])
            if correlation > 0.5:
                correlation_type = "Positive" if correlation > 0 else "Negative"
                correlation_data.append({'column1': column1, 'column2': column2, 'correlation_type': correlation_type, 'correlation_value': correlation})
                print(f'{correlation_type} Correlation found: {column1} vs {column2} ({correlation})')                
                plt.figure()
                scatter = plt.scatter(full_data[column1], full_data[column2], c=full_data['encoded_planting_number'], cmap=cmap, norm=mcolors.BoundaryNorm(np.arange(-0.5, num_colors, 1), cmap.N))
                plt.colorbar(scatter, ticks=np.arange(0, num_colors, 1), label='Planting Number')
                plt.xlabel(column1)
                plt.ylabel(column2)
                plt.title(f'{column1} vs {column2}. Pearson: {correlation}')
                plt.show()

            elif correlation < -0.5:
                correlation_type = "Positive" if correlation > 0 else "Negative"
                correlation_data.append({'column1': column1, 'column2': column2, 'correlation_type': correlation_type, 'correlation_value': correlation})
                print(f'{correlation_type} Correlation found: {column1} vs {column2} ({correlation})')
                plt.figure()
                scatter = plt.scatter(full_data[column1], full_data[column2], c=full_data['encoded_planting_number'], cmap=cmap, norm=mcolors.BoundaryNorm(np.arange(-0.5, num_colors, 1), cmap.N))
                plt.colorbar(scatter, ticks=np.arange(0, num_colors, 1), label='Planting Number')
                plt.xlabel(column1)
                plt.ylabel(column2)
                plt.title(f'{column1} vs {column2}. Pearson: {correlation}')
                plt.show()

correlation_df = pd.DataFrame(correlation_data)
correlation_df.to_csv('',index=True)

In [ ]:
#Table of significant correlations found in all dataset
correlation_df

In [ ]:
#Identification and visualization of significant variable relationship using linear regression
float_columns = full_data.select_dtypes(include=['float64']).columns
column_combinations = list(itertools.combinations(float_columns, 2))
column_combinations_filtered = [comb for comb in column_combinations if 'IND_Plant_ID' not in comb 
                                and full_data[comb[0]].notnull().any() 
                                and full_data[comb[1]].notnull().any()]
regression_data = []

for column1, column2 in column_combinations_filtered:
    X = full_data[[column1, column2]]
    X = sm.add_constant(X)
    y = full_data[column2]
    model = sm.OLS(y, X, missing='drop').fit()
    
    if 0.0 < model.pvalues.iloc[1] < 0.05:  
        pvalue = model.pvalues.iloc[1]
        print(f"Regression Results: {column2} as response variable and {column1} as predictor variable with p-value of {pvalue}")        
        regression_data.append({'column1': column1, 'column2': column2, 'coefficient': model.params[1], 'std_err':model.bse[1],'t_value': model.tvalues[1], 'p_value':model.pvalues[1]})
        sns.lmplot(x=column1, y=column2, data=full_data, hue='encoded_planting_number', aspect=1.5, fit_reg=False, palette='viridis')
        plt.title(f'{column2} as response variable and {column1} as predictor variable')
        plt.show()
        print(model.summary())
regression_df = pd.DataFrame(regression_data)
regression_df.rename(columns={'column1': 'predictor', 'column2': 'response'}, inplace=True)
regression_df = regression_df.sort_values(by='std_err', ascending=False)

In [ ]:
regression_df = regression_df.sort_values(by='std_err', ascending=True).reset_index(drop=True)
regression_df.head(20)


In [ ]:
correlation_df.to_csv('correlation_result.csv', index=False)

In [ ]:
full_data.info()